In [1]:
import pandas as pd
import numpy as np
import pickle
from tqdm.notebook import tqdm
from glove import Corpus
from mittens import GloVe

In [2]:
filename = "../data/train/train_dataset.csv"

In [3]:
data = pd.read_csv(filename, header=None, names=['text','label'])

In [4]:
'''
    Create list of sentences
'''
sentences = []
sentence = []

for word in data.text:
    if type(word)!=float:
        sentence.append(word)
    else:
        sentences.append(' '.join(sentence))
        sentence = []

In [5]:
def read_corpus(sentences):
    '''
    Read corpus from list (generator)
    '''
    for line in sentences:
        yield line.lower().split(' ')

In [6]:
# read corpus
get_data = read_corpus(sentences)
# create co-occurence matrix
corpus_model = Corpus()
corpus_model.fit(get_data, window=10)
# create word2idx from corpus
word2idx_corpus = sorted(corpus_model.dictionary.items(), key=lambda x: x[1])

In [7]:
# Load `cooccurrence`
cooccurrence = corpus_model.matrix.toarray()
# embedding dimension is 300, iteration is 5000
glove_model = GloVe(n=300, max_iter=5000) 
embeddings = glove_model.fit(cooccurrence)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Iteration 5000: loss: 4.5993299484252935

In [8]:
pickle.dump(word2idx_corpus, open("../data/pickle_file/word2idx_corpus.pkl", "wb"))

In [9]:
filename = '../model/glove_embedding.pkl'
pickle.dump(embeddings, open(filename, "wb"))